# Chapter 2: Hidden Threads - Mastering the Art of Steganography in Rust

## Introduction

At the core of this fascinating chapter lies the perplexing art of [**steganography**](https://en.wikipedia.org/wiki/Steganography), a term derived from the fusion of two ancient Greek words - "steganos", meaning "to cover or protect," and "graphien", signifying "to write." In the vast domain of [**cybersecurity**](https://en.wikipedia.org/wiki/Computer_security), steganography serves as a captivating technique, like a digital cover, hiding valuable data within the seemingly innocent façade of images. This practice has become routine within the security community, a skill where practitioners adeptly navigate the delicate balance between hiding and disclosure. In essence, steganography empowers security professionals to embed payloads secretly, patiently awaiting the opportune moment for extraction once the data reaches its intended destination.

As we kick off this chapter through the complexities of steganography, our focus sharpens on the security landscape, where the art of hiding and revealing data takes center stage. The essence of this practice lies in the ingenious hiding of information within the very fabric of images. Like a hidden message within a painting, steganography allows security practitioners to obscure critical payloads within the pixels of images, laying the groundwork for secret communication. This chapter, which is like a roadmap through the hidden passages of digital secrecy, reveals the techniques and procedures employed in this confidential art, offering insights into the methods that security professionals employ to navigate this delicate dance of data manipulation.

The chapter, like a guidebook for digital spies, delves into the practical aspects of this obscuring art, particularly focusing on the embedding of data within [**Portable Network Graphics (PNG)**](https://en.wikipedia.org/wiki/PNG) images. The PNG format, known for its ubiquity and versatility, becomes the canvas upon which the steganographer paints hidden messages. The journey through this chapter promises a deep dive into the complexities of PNG files, exploring their byte structure, decoding headers, and cracking the sequence of chunks that define the anatomy of these digital canvases. In essence, the chapter stands as a beacon for those navigating the maze of steganography, offering both theoretical insights and practical wisdom for concealing and unveiling secrets within the digital realm.

## Table of Contents

* [**Exploring the PNG Format**](#1.-Exploring-the-PNG-Format)
* [**Reading a PNG File**](#2.-Reading-a-PNG-File)

## 1. Exploring the PNG Format

To kick off our journey of embedding data within the PNG format, we must first understand the complex byte curtain that constitutes a binary PNG image file. The PNG specification, accessible at [**libpng.org**](http://www.libpng.org/pub/png/spec/1.2/PNG-Structure.html#PNG-file-signature), becomes our compass in navigating this digital landscape. Within this knowledge, byte chunks incorporate the fabric of PNG images, creating a canvas where our steganographic secrets will find their hiding places. These chunks, repetitive in nature, lay the groundwork for our exploration into the hidden domains of data manipulation.

**The Enigmatic Header:**

![ScreenShot taken from the Bless hex editor](https://dev-to-uploads.s3.amazonaws.com/uploads/articles/m9y570ydqf6j6f882lwf.png)

In the realm of PNG files, our journey begins with the opening eight bytes, forming the magical sequence "**89 50 4E 47 0D 0A 1A 0A**". This sequence acts as a secret code, the key that unlocks the PNG world. Picture it like the first musical notes of our steganographic symphony. Now, onto the scene comes the `Header` struct, a kind of container designed to catch these magical bytes.

```rust
// Header holds the first UINT64 (Magic Bytes)
struct Header {
    header: u64,
}
```

The `Header` structure is like an artist's canvas waiting for a masterpiece. It holds an unsigned 64-bit integer, a perfect fit for the magical bytes. When we perform a magical act called [**transmutation**](https://www.google.com/search?channel=fs&client=ubuntu-sn&q=Transmute+rust+), the hidden message "PNG" is revealed, almost like a secret handshake granting us access to the **PNG** world. This Header is our guardian, making sure our PNG image is the real deal and giving us the green light to start our work of steganography. It's like the keeper of secrets, ensuring the symphony begins without any interference.

**Decoding the Chunk Sequence:**

As we venture beyond the surface of the PNG image, we encounter something known as the chunk sequence. This sequence follows structured data of **SIZE** (4 bytes), **TYPE** (4 bytes), **DATA** (variable bytes), and **CRC** (4 bytes). Each chunk is like a puzzle piece in the steganographic mosaic and contributes its unique narrative to the overall story painted within the **PNG** image. The **SIZE** chunk sets the stage, dictating the length of the upcoming **DATA**, while the **TYPE** chunk acts as a sign, revealing the purpose of the data that follows. Our mission is to decode this sequence, uncovering the hidden messages that lie beneath.

```rust
// Chunk represents a data byte chunk segment
struct Chunk {
    size: u32,
    r#type: u32,
    data: Vec<u8>,
    crc: u32,
}
```

As we embark on the complexities of steganography within the PNG image, the `MetaChunk` structure emerges as our reliable companion, illuminating the path through the concealed landscape of digital storytelling. Picture the `MetaChunk` as our guide, equipped with the tools to decode the complexities hidden within each segment of our steganographic work.

```rust
// MetaChunk structure orchestrating the steganographic journey
struct MetaChunk {
    header: Header,
    chk: Chunk,
    offset: u64,
}
```

The `MetaChunk` will navigate us through the hidden landscapes of binary complexities. Its fields, such as the `header` and `chk` (representing the Header and Chunk structures, respectively), are our compass and map, ensuring we stay on course through this steganographic work. The `offset`, is like a GPS coordinate, pinpoints our location within the byte landscape, marking the starting point of each chunk segment.

## 2. Reading a PNG File

**Preprocessing the Image:**
The `pre_process_image` function, our initiation into the PNG world, transforms the PNG image into a readable script. It's the first step in turning a PNG image, which seems like a puzzle, into something we can read and work with. This function does some important things to make that happen.

```rust
// Implementation of MetaChunk, defining a function `pre_process_image` that processes a PNG image file
impl MetaChunk {
    fn pre_process_image(file: &mut File) -> Result<MetaChunk, std::io::Error> {
        // Creating a Header struct to hold the first 8 bytes of the PNG image
        let mut header = Header { header: 0 };
        // Reading exactly 8 bytes from the file into the header using unsafe operations
        file.read_exact(unsafe { mem::transmute::<_, &mut [u8; 8]>(&mut header.header) })?;

        // Converting the header bytes from u64 to u8 array
        let b_arr = u64_to_u8_array(header.header);

        // Checking if the PNG magic bytes are present in the file
        if &b_arr[1..4] != b"PNG" {
            // If not, panicking and terminating the program with an error message
            panic!("Provided file is not a valid PNG format");
        } else {
            // If yes, printing a confirmation message
            println!("It is a valid PNG file. Let's process it!");
        }

        // Getting the current position (offset) in the file and storing it
        let offset = file.seek(SeekFrom::Current(0))?;

        // Returning a `MetaChunk` struct instance with the obtained header, an initial Chunk, and the offset
        Ok(MetaChunk {
            header,
            chk: Chunk {
                size: 0,
                r#type: 0,
                data: Vec::new(),
                crc: 0,
            },
            offset,
        })
    }
}
```

Once initiated, validation becomes crucial. This method inspect the first eight bytes, ensuring their adherence to the PNG format. The magic bytes unfold, and if the decoded message aligns with "PNG", the gateway to our steganographic exploration swings open.

**Reading through Chunk Sequences:**

![ScreenShot taken from the Bless hex editor](https://dev-to-uploads.s3.amazonaws.com/uploads/articles/33kasxf4cstfnq5ak2l8.png)

Our journey into the PNG landscape is marked by the `process_image` function, a blend of code orchestrating the navigation through chunks. A loop, a recurring motif, guides the traversal through each chunk. The `count` becomes our companion, marking each chunk's unveiling in this steganographic manuscript.

```rust
fn process_image(&mut self, file: &mut File) {
    let mut count = 1;
    let mut chunk_type = String::new();
    let end_chunk_type = "IEND";
    
    while chunk_type != end_chunk_type {
        println!("---- Chunk # {} ----", count);
        let offset = self.get_offset(file);
        println!("Chunk Offset: {:x}", offset);
        self.read_chunk(file);
        chunk_type = self.chunk_type_to_string();
        count += 1;
    }
}
```

**Charting Coordinates with Offset:**

In this exploration, awareness of our position within the PNG image becomes imperative. The `get_offset` method, a virtual compass, captures the coordinates of each chunk. The [**`seek`**](https://doc.rust-lang.org/std/io/trait.Seek.html#tymethod.seek) function, a navigator's tool, unfolds the map of offsets, guiding our steganographic ship through the PNG terrain. It moves to an offset, in bytes, in the file stream.

```rust
fn get_offset(&mut self, data: &mut File) -> u64 {
    let offset = data.seek(SeekFrom::Current(0)).unwrap();
    self.offset = offset;
    offset
}
```

**Deciphering Chunk by Chunk:**

The heart of our steganographic novel lies in the `read_chunk` method—a meticulous deciphering of each chunk's data. The synchronization of `read_chunk_size`, `read_chunk_type`, `read_chunk_bytes`, and `read_chunk_crc` becomes a choreography of bytes, each revealing a distinct aspect of the hidden bytes.

```rust
fn read_chunk(&mut self, data: &mut File) {
    self.read_chunk_size(data);
    self.read_chunk_type(data);
    self.read_chunk_bytes(data, self.chk.size);
    self.read_chunk_crc(data);
}
```

**Transcribing the Size Chunk:**

The `read_chunk_size` method reads the size of each chunk. The `size_bytes` variable holds the essence of the chunk size on the paper of our steganographic manuscript.

```rust
fn read_chunk_size(&mut self, data: &mut File) {
    let mut size_bytes = [0; 4];

    match data.read_exact(&mut size_bytes) {
        Ok(_) => {
            self.chk.size = u32::from_be_bytes(size_bytes);
        }
        Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
            eprintln!("Warning: Reached end of file prematurely while reading chunk size");
        }
        Err(err) => {
            eprintln!("Error reading chunk size bytes: {}", err);
        }
    }
}
```

**Decoding the Chunk Type:**
The `read_chunk_type` method deciphers the type of each chunk.

```rust
fn read_chunk_type(&mut self, data: &mut File) {
    let mut type_bytes = [0; 4];

    match data.read_exact(&mut type_bytes) {
        Ok(_) => {
            self.chk.r#type = u32::from_be_bytes(type_bytes);
        }
        Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
            eprintln!("Warning: Reached end of file prematurely while reading chunk type");
        }
        Err(err) => {
            eprintln!("Error reading chunk type bytes: {}", err);
        }
    }
}
```

**Reading the Chunk's Bytes:**
The `read_chunk_bytes` method reads each chunk of the image file given a number of bytes.

```rust
fn read_chunk_bytes(&mut self, data: &mut File, len: u32) {
    self.chk.data = vec![0; len as usize];

    match data.read_exact(&mut self.chk.data) {
        Ok(_) => {
            // Successfully read the expected number of bytes
        }
        Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
            eprintln!("Error reading chunk bytes: Reached end of file prematurely");
            self.chk
                .data
                .truncate(data.seek(SeekFrom::Current(0)).unwrap() as usize);
        }
        Err(err) => {
            eprintln!("Error reading chunk bytes: {}", err);
        }
    }
}
```

**Decoding the Chunk's CRC:**
The `read_chunk_crc` method takes on the role of the guardian, deciphering the script's integrity. The CRC bytes ensure that the essence of the chunk remains unaltered.

```rust
fn read_chunk_crc(&mut self, data: &mut File) {
    let mut crc_bytes = [0; 4];

    match data.read_exact(&mut crc_bytes) {
        Ok(_) => {
            self.chk.crc = u32::from_be_bytes(crc_bytes);
        }
        Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
            eprintln!("Warning: Reached end of file prematurely while reading chunk CRC");
        }
        Err(err) => {
            eprintln!("Error reading chunk CRC bytes: {}", err);
        }
    }
}
```

As the stenographic odyssey unfolds, each line of code becomes a step in deciphering the hidden canvas of PNG images. The orchestration of structs, functions, and bytes creates a symphony that resonates with the heartbeat of steganography. With every offset, chunk, and byte, the cover over PNG's secrets slowly lifts, revealing a digital narrative waiting to be explored.

Now, let's put it all together.

In [3]:
use std::fs::File;
use std::io::ErrorKind;
use std::io::{Read, Seek, SeekFrom};
use std::mem;
use std::str;

#[derive(Debug)]
struct Header {
    header: u64,
}

#[derive(Debug)]
struct Chunk {
    size: u32,
    r#type: u32,
    data: Vec<u8>,
    crc: u32,
}

struct MetaChunk {
    header: Header,
    chk: Chunk,
    offset: u64,
}

fn u64_to_u8_array(value: u64) -> [u8; 8] {
    let bytes = value.to_ne_bytes();
    let mut result = [0; 8];

    unsafe {
        result = mem::transmute_copy(&bytes);
    }

    result
}

impl MetaChunk {
    fn pre_process_image(file: &mut File) -> Result<MetaChunk, std::io::Error> {
        let mut header = Header { header: 0 };
        file.read_exact(unsafe { mem::transmute::<_, &mut [u8; 8]>(&mut header.header) })?;

        let b_arr = u64_to_u8_array(header.header);
        if &b_arr[1..4] != b"PNG" {
            panic!("Not a valid PNG format");
        } else {
            println!("It is a valid PNG file. Let's process it!");
        }

        let offset = file.seek(SeekFrom::Current(0))?;
        Ok(MetaChunk {
            header,
            chk: Chunk {
                size: 0,
                r#type: 0,
                data: Vec::new(),
                crc: 0,
            },
            offset,
        })
    }

    fn process_image(&mut self, file: &mut File) {
        let mut count = 1;
        let mut chunk_type = String::new();
        let end_chunk_type = "IEND";
        
        while chunk_type != end_chunk_type {
            println!("---- Chunk # {} ----", count);
            let offset = self.get_offset(file);
            println!("Chunk Offset: {:x}", offset);
            self.read_chunk(file);
            chunk_type = self.chunk_type_to_string();
            count += 1;
        }
    }

    fn get_offset(&mut self, file: &mut File) -> u64 {
        let offset = file.seek(SeekFrom::Current(0)).unwrap();
        self.offset = offset;
        offset
    }

    fn read_chunk(&mut self, file: &mut File) {
        self.read_chunk_size(file);
        self.read_chunk_type(file);
        self.read_chunk_bytes(file, self.chk.size);
        self.read_chunk_crc(file);
    }

    fn read_chunk_size(&mut self, file: &mut File) {
        let mut size_bytes = [0; 4];

        match file.read_exact(&mut size_bytes) {
            Ok(_) => {
                // Successfully read the expected number of bytes
                self.chk.size = u32::from_be_bytes(size_bytes);
            }
            Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
                // Handle the situation where the file ends before reading the expected bytes
                eprintln!("Warning: Reached end of file prematurely while reading chunk size");
            }
            Err(err) => {
                eprintln!("Error reading chunk size bytes: {}", err);
            }
        }
    }

    fn read_chunk_type(&mut self, file: &mut File) {
        let mut type_bytes = [0; 4];

        match file.read_exact(&mut type_bytes) {
            Ok(_) => {
                // Successfully read the expected number of bytes
                self.chk.r#type = u32::from_be_bytes(type_bytes);
            }
            Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
                // Handle the situation where the file ends before reading the expected bytes
                eprintln!("Warning: Reached end of file prematurely while reading chunk type");
            }
            Err(err) => {
                eprintln!("Error reading chunk type bytes: {}", err);
            }
        }
    }

    fn read_chunk_bytes(&mut self, file: &mut File, len: u32) {
        self.chk.data = vec![0; len as usize];

        match file.read_exact(&mut self.chk.data) {
            Ok(_) => {
                // Successfully read the expected number of bytes
            }
            Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
                eprintln!("Error reading chunk bytes: Reached end of file prematurely");
                // Update the length of the Chunk based on the actual number of bytes read
                self.chk
                    .data
                    .truncate(file.seek(SeekFrom::Current(0)).unwrap() as usize);
            }
            Err(err) => {
                eprintln!("Error reading chunk bytes: {}", err);
            }
        }
    }

    fn read_chunk_crc(&mut self, file: &mut File) {
        let mut crc_bytes = [0; 4];

        match file.read_exact(&mut crc_bytes) {
            Ok(_) => {
                // Successfully read the expected number of bytes
                self.chk.crc = u32::from_be_bytes(crc_bytes);
            }
            Err(err) if err.kind() == ErrorKind::UnexpectedEof => {
                // Handle the situation where the file ends before reading the expected bytes
                eprintln!("Warning: Reached end of file prematurely while reading CRC");
            }
            Err(err) => {
                eprintln!("Error reading CRC bytes: {}", err);
            }
        }
    }

    fn chunk_type_to_string(&self) -> String {
        String::from_utf8_lossy(&self.chk.r#type.to_be_bytes()).to_string()
    }
}

let mut file = File::open("stegano/prj.png").expect("Error opening file");

let mut meta_chunk = MetaChunk::pre_process_image(&mut file).expect("Error processing image");

meta_chunk.process_image(&mut file);

It is a valid PNG file. Let's process it!
---- Chunk # 1 ----
Chunk Offset: 8
---- Chunk # 2 ----
Chunk Offset: 21
---- Chunk # 3 ----
Chunk Offset: 2e
---- Chunk # 4 ----
Chunk Offset: 203a
---- Chunk # 5 ----
Chunk Offset: 4046
---- Chunk # 6 ----
Chunk Offset: 6052
---- Chunk # 7 ----
Chunk Offset: 805e
---- Chunk # 8 ----
Chunk Offset: a06a
---- Chunk # 9 ----
Chunk Offset: c076
---- Chunk # 10 ----
Chunk Offset: e082
---- Chunk # 11 ----
Chunk Offset: 1008e
---- Chunk # 12 ----
Chunk Offset: 1209a
---- Chunk # 13 ----
Chunk Offset: 140a6
---- Chunk # 14 ----
Chunk Offset: 160b2
---- Chunk # 15 ----
Chunk Offset: 180be
---- Chunk # 16 ----
Chunk Offset: 1a0ca
---- Chunk # 17 ----
Chunk Offset: 1c0d6
---- Chunk # 18 ----
Chunk Offset: 1e0e2
---- Chunk # 19 ----
Chunk Offset: 200ee
---- Chunk # 20 ----
Chunk Offset: 220fa
---- Chunk # 21 ----
Chunk Offset: 24106
---- Chunk # 22 ----
Chunk Offset: 26112
---- Chunk # 23 ----
Chunk Offset: 26d33


![ScreenShot taken from the Bless hex editor](https://dev-to-uploads.s3.amazonaws.com/uploads/articles/ns2mwc5fe14k677iq0ws.png)

The output signifies the successful validation of the PNG file as indicated by the confirmation message, "It is a valid PNG file. Let's process it!" This message is printed after checking if the magic bytes "PNG" are present in the file.

Following this confirmation, the program proceeds to process the PNG file in a chunk-wise manner. Each line in the output corresponds to a different chunk within the PNG file, and the offset value indicates the position of each chunk in hexadecimal format. The term "Chunk Offset" is essentially revealing the starting point of each chunk within the file.

For instance, the first chunk (Chunk #1) starts at an offset of 8, the second chunk (Chunk #2) at an offset of 21, and so forth. These offsets provide a crucial reference point for navigating and manipulating the image file. In the context of steganography, understanding the position of each chunk becomes essential for hiding and extracting hidden data without corrupting the file structure.

Having meticulously traversed the header of the PNG image and comprehensively processed it in a chunk-wise fashion, we have gained invaluable insights into the complex structure of the image file. The journey has developed as a meticulous exploration of the PNG format, unraveling the magic bytes and delving into the sequential chunks that compose the image.

Now armed with the knowledge of how to navigate through the chunks and decipher their offsets, our next work is to embark on the hiding of a message within these layers of data. This entails the artful insertion of our payload into the file, ensuring its secret existence within the seemingly innocent image structure.